Battle Plan:






Done:
- allow easier python specdefinition
- %reg syntax for cool inline asm demos
- 



def add_subst
def add_state_var
def add_cb


syscalls?


Interrupt preludes
It got a lot slower?
pi pico
More examples from frama-c, s2n-bignum
simd
reads/writes
kd_always. Some interrupt bug?

Change register names to enable true inline assembly.

Can I use unicorn to validate against that easily?


spacer? I'd be shocked if useful.



sizex <= x - y <= -sizey \/  sizey <= y - x <= -sizex

We can say exactly when the or happens
if x > y 
then
    sizey <= x - y <= -sizex
else
    sizey <= y - x <= -sizex

inequality and addition / diff son't play the same as the ints.

https://mcyoung.xyz/2023/11/27/simd-base64/  simd

Would I want to state from particular entry? or even address

kd_reachable from_label here_label
kd_reachable here_label
kd_unreachable from_label here_label

address expressions. Literal address for label, or `<foo+0x7>` relative addressing. I guess not necessary if this is mainly for textual assembly we have no control over. In that case just use programmatic bindings

https://github.com/toolCHAINZ/jingle


https://github.com/m4b/goblin  An impish, cross-platform binary parsing crate, written in Rust
https://github.com/m4b/bingrep
https://github.com/m4b/faerie


https://ruor.uottawa.ca/server/api/core/bitstreams/86cc22d7-4517-4744-9cf7-4b1745a6c029/content race conditions and memory models. AMusing because that _is_ what sail is for. Could be fun.

https://news.ycombinator.com/item?id=40185065 https://alastairreid.github.io/riscv-spec-issues/
https://github.com/riscv/riscv-opcodes just the opcodes

https://cs.ru.nl/~mschool/pub/vstte-why3-avr-revised.pdf why3 avr

https://antmicro.com/blog/2025/09/girdl-ghidra-renode-integration/ https://github.com/antmicro/girdl pull in svd files? That's a good idea. I doubt I can directly use this

In [ ]:
kd_prelude "(define-fun nonoverlap ((a (_ BitVec 64)) (sa (_ BitVec 64)) (b (_ BitVec 64)) (sb (_ BitVec 64))))"
kd_prelude "(define-const bvone (_ bv1 64))"
kd_prelude "(define-const bvtwo (_ bv2 64))"
kd_prelude "(define-fun select32 ...)"

In [18]:
from kdrag.all import *

x,y = smt.BitVecs("x y", 64)
sizex, sizey = smt.Ints("sizex, sizey")
bsizex = smt.Int2BV(sizex, 64)
bsizey = smt.Int2BV(sizey, 64)
maxint = smt.BitVecVal(2**64-1, 64)
no_overlap = kd.define("no_overlap", [x,sizex,y,sizey], 
    smt.If( smt.ULE(x,y),
       smt.And( smt.ULT(bsizex , y - x), smt.ULE(bsizey, maxint - (y - x))),
       smt.And( smt.ULT(bsizey , x - y), smt.ULE(bsizex, maxint - (x - y)))
  ))

kd.prove(smt.Implies(smt.Distinct(x, x+1, y, y+1, y+2), no_overlap(x,2,y,3)), unfold=1)



LemmaError: (Implies(Distinct(x, x + 1, y, y + 1, y + 2),
        no_overlap(x, 2, y, 3)), 'Countermodel', [y = 18446744073709551613,
 x = 18446744073709551611,
 no_overlap = [else -> False]])

In [17]:
2**64-1

18446744073709551615

# TraceFragmentBundle

It's interesting to be able to run code to use in specs

list[TraceBundleFragment] could be the return type of run_all_paths. Collected up by common entry or




In [ ]:
@dataclass
class TraceFragmentBundle:
    """
    A bundle is a set of pathways
    """
    start_addr : int # all at same start addr? Not necesarily all end at same exit addr
    ctx : pcode.BinaryContext
    args : # or is args a mapping from initial ghost state? #list[smt.ExprRef] # ghost vars?
    vcs: list[VerificationCondition]
    rets : list[smt.ExprRef] # ghost vars?

    @staticmethod
    def from_entry_exit(ctx: pcode.BinaryContext, entry_addr: int, exit_addr: int):
        spec = AsmSpec()
        spec.add_entry("entry", entry_addr, smt.BoolVal(True))
        spec.add_exit("exit", exit_addr, smt.BoolVal(True))
        return TraceFragmentBundle(ctx, run_all_paths(ctx, spec))

    def __call__(self, memstate : pcode.MemState) -> smt.BoolRef:
        """
        The bundle considered as a Function.
        Joined together.

        """
        # But a function from ghost to ghost or from memstate to memstate? or memstate to ghost?
        acc = memstate.Const("undef", bits=ctx.bits)
        for vc in vcs:
            smt.If(vc.path_cond, vc.memstate, acc)
        return acc

# compile aexpr

type aexpr = I64 i64 | Add aexpr aexpr
eval(compile(e)) == 

Or even a model of some kind of hack

virtual machine


I kind of did the conrete version


But the btecode interpreter version is interesting.






# memset
interpreted memset

Swtich on type is also intereting.
Interpreter


Is this overwrought compared to what we have to do though?

type MemLang = SetList list (addr * value) | SetMany int addr value (* iterate the construction *)

do finite  unroll

Compare to my high level versions? Output of ocmpiler
oct 7 demo

get riscv32 toolchain in ci

get small snippets






In [130]:
%%file /tmp/mymemset.s

# kd_entry mymemset "true"

.globl mymemset
mymemset:
    lw t1, 0(a0) # load dst ptr
    lw t2, 4(a0) # load value
    lw t3, 8(a0) # load length
    sw t2, 0(t1) # store value
    ret

Overwriting /tmp/mymemset.s


In [131]:
! nix-shell -p pkgsCross.riscv32-embedded.buildPackages.gcc --run "riscv32-none-elf-as /tmp/mymemset.s -o /tmp/mymemset.o && riscv32-none-elf-objdump -d /tmp/mymemset.o"


/tmp/mymemset.o:     file format elf32-littleriscv


Disassembly of section .text:

00000000 <mymemset>:
   0:	00052303          	lw	t1,0(a0)
   4:	00452383          	lw	t2,4(a0)
   8:	00852e03          	lw	t3,8(a0)
   c:	00732023          	sw	t2,0(t1)
  10:	00008067          	ret


In [ ]:
import kdrag.contrib.pcode as pcode
import kdrag.contrib.pcode.asmspec as asmspec
import kdrag.smt as smt
spec = asmspec.AsmSpec()
ctx = ctx = pcode.BinaryContext("/tmp/mymemset.o", langid="RISCV:LE:32:default")
startaddr = ctx.loader.find_symbol("mymemset").rebased_addr

# TODO: actually this is something that should be exposed.
ram32 = ctx._subst_decls['ram32']
t0 = ctx._subst_decls['t0']
a0 = ctx._subst_decls['a0']


spec.add_entry("mymemset",  startaddr , smt.BoolVal(True))
spec.add_assign("mymemset", startaddr, "dst", ram32[a0])
spec.add_assign("mymemset", startaddr, "value", ram32[a0 + 4])
dst, value = smt.BitVecs('dst value', 32)
spec.add_exit("myexit", startaddr + 16, ram32[dst] == value)

vcs = asmspec.run_all_paths(ctx, spec)
for vc in vcs:
    #print(vc)
    # ok ctx.substitute is very slow because gathering constants is slow. Worth persuing?
    vc.verify(ctx)


conmsts
don
Executing SpecStmt: Assign(label='mymemset', addr=4194304, name='dst', expr=ram32[a0])
conmsts
don
Executing SpecStmt: Assign(label='mymemset', addr=4194304, name='value', expr=ram32[a0 + 4])
conmsts
don
Executing 0x400000/4: lw t1,0x0(a0) at (4194304, 0) PCODE IMARK ram[400000:4]
Executing 0x400000/4: lw t1,0x0(a0) at (4194304, 1) PCODE unique[580:4] = 0x0
Executing 0x400000/4: lw t1,0x0(a0) at (4194304, 2) PCODE unique[3080:4] = a0 + unique[580:4]
Executing 0x400000/4: lw t1,0x0(a0) at (4194304, 3) PCODE t1 = *[ram]unique[3080:4]
Executing 0x400004/4: lw t2,0x4(a0) at (4194308, 0) PCODE IMARK ram[400004:4]
Executing 0x400004/4: lw t2,0x4(a0) at (4194308, 1) PCODE unique[580:4] = 0x4
Executing 0x400004/4: lw t2,0x4(a0) at (4194308, 2) PCODE unique[3080:4] = a0 + unique[580:4]
Executing 0x400004/4: lw t2,0x4(a0) at (4194308, 3) PCODE t2 = *[ram]unique[3080:4]
Executing 0x400008/4: lw t3,0x8(a0) at (4194312, 0) PCODE IMARK ram[400008:4]
Executing 0x400008/4: lw t3,0x8(a0) at

In [3]:
%%file /tmp/mymemcpy.c
#include <stddef.h>
void mymemcpy(int *dst, const int *src, size_t n) {
    for (size_t i = 0; i < n; i++) {
        dst[i] = src[i];
    }
    return;
}

Writing /tmp/mymemcpy.c


In [6]:
! nix-shell -p pkgsCross.riscv32-embedded.buildPackages.gcc --run "riscv32-none-elf-gcc -O1 -c /tmp/mymemcpy.c -o /tmp/mymemcpy.o && riscv32-none-elf-objdump -d /tmp/mymemcpy.o"


/tmp/mymemcpy.o:     file format elf32-littleriscv


Disassembly of section .text:

00000000 <mymemcpy>:
   0:	ca11                	beqz	a2,14 <.L1>
   2:	87ae                	mv	a5,a1
   4:	060a                	slli	a2,a2,0x2
   6:	95b2                	add	a1,a1,a2

00000008 <.L3>:
   8:	4398                	lw	a4,0(a5)
   a:	c118                	sw	a4,0(a0)
   c:	0791                	addi	a5,a5,4
   e:	0511                	addi	a0,a0,4
  10:	feb79ce3          	bne	a5,a1,8 <.L3>

00000014 <.L1>:
  14:	8082                	ret


In [9]:
import kdrag.contrib.pcode as pcode
import kdrag.contrib.pcode.asmspec as asmspec
import kdrag.smt as smt
spec = asmspec.AsmSpec()
ctx = ctx = pcode.BinaryContext("/tmp/mymemcpy.o", langid="RISCV:LE:32:default")
startaddr = ctx.loader.find_symbol("mymemcpy").rebased_addr

# TODO: actually this is something that should be exposed.
ram32 = ctx._subst_decls['ram32']
dstptr = ctx._subst_decls['a0']
srcptr = ctx._subst_decls['a1']
n = ctx._subst_decls['a2']

#  "(and (= RDI (_ bv0 64)) (= RSI (_ bv1024 64)) (bvuge RDX (_ bv0 64)) (bvult RDX (_ bv32 64)))"
spec.add_entry("mymemcpy",  startaddr , smt.And(n == 0))
# kd_cut mycut "(and (= RDI (_ bv0 64)) (= RSI (_ bv1024 64)) (bvuge RDX (_ bv0 64)) (bvult RDX (_ bv32 64)) (bvult RAX RDX) 
# (forall ((offset (_ BitVec 64))) (=> (and (bvuge offset (_ bv0 64)) (bvult offset RAX)) (= (select ram32 (bvadd RDI (bvshl offset (_ bv2 64)))) 
# (select ram32 (bvadd RSI (bvshl offset (_ bv2 64))))))))"

#spec.add_cut("loopstart", startaddr + 8, smt.And(True))


# kd_exit myexit "(=> (and (bvuge offset (_ bv0 64)) (bvult offset RDX)) (= (select ram32 (bvadd RDI (bvshl offset (_ bv2 64)))) 
# (select ram32 (bvadd RSI (bvshl offset (_ bv2 64))))))"
spec.add_exit("mymemset_exit", startaddr + 14, smt.And(True))

vcs = asmspec.run_all_paths(ctx, spec)
for vc in vcs:
    #print(vc)
    # ok ctx.substitute is very slow because gathering constants is slow. Worth perusing?
    vc.verify(ctx)


conmsts
don
Executing 0x400000/2: c.beqz a2,0x400014 at (4194304, 0) PCODE IMARK ram[400000:2]
Executing 0x400000/2: c.beqz a2,0x400014 at (4194304, 1) PCODE unique[10800:1] = a2 == 0x0
Executing 0x400000/2: c.beqz a2,0x400014 at (4194304, 2) PCODE if (unique[10800:1]) goto ram[400014:4]
Executing 0x400014/2: ret  at (4194324, 0) PCODE IMARK ram[400014:2]
Executing 0x400014/2: ret  at (4194324, 1) PCODE return ra


NotImplementedError: ('Opcode not implemented: ', OpCode.RETURN)

Track better which trace fragment we're on.
Trace bundles


In [150]:
ctx._subst_decls['t0']

t0

In [112]:
%%file /tmp/tag.c

enum Op {Inc1, Inc2, Inc3};

int doop(enum Op op, int x) {
    switch(op) {
        case Inc1: return x + 1;
        case Inc2: return x + 2;
        case Inc3: return x + 3;
    }
    return -1;
}




Overwriting /tmp/tag.c


In [115]:
! nix-shell -p pkgsCross.riscv32-embedded.buildPackages.{gcc,binutils} --run "riscv32-none-elf-gcc -O2 -S /tmp/tag.c -o /tmp/tag.s && cat /tmp/tag.s"

	.file	"tag.c"
	.option nopic
	.attribute arch, "rv32i2p1_m2p0_a2p1_f2p2_d2p2_c2p0_zicsr2p0_zifencei2p0"
	.attribute unaligned_access, 0
	.attribute stack_align, 16
	.text
	.align	1
	.globl	doop
	.type	doop, @function
doop:
	li	a4,1
	mv	a5,a0
	beq	a0,a4,.L2
	li	a4,2
	addi	a0,a1,3
	beq	a5,a4,.L1
	li	a0,-1
	bne	a5,zero,.L1
	addi	a0,a1,1
	ret
.L2:
	addi	a0,a1,2
.L1:
	ret
	.size	doop, .-doop
	.ident	"GCC: (GNU) 13.3.0"


In [110]:
! ! /home/philip/Downloads/lowrisc-toolchain-gcc-rv32imcb-x86_64-20250710-1/bin/riscv32-unknown-elf-gcc -W -c -S -o /tmp/tag.s -O1   /tmp/tag.c

In [ ]:
! cat /tmp/tag.s 

	.file	"tag.c"
	.option nopic
	.attribute arch, "rv32i2p0_m2p0_c2p0"
	.attribute unaligned_access, 0
	.attribute stack_align, 16
	.text
	.align	1
	.globl	doop
	.type	doop, @function
doop:
	mv	a5,a0
	li	a4,1
	beq	a0,a4,.L2
	li	a4,2
	addi	a0,a1,3
	beq	a5,a4,.L1
	li	a0,-1
	beq	a5,zero,.L6
.L1:
	ret
.L6:
	addi	a0,a1,1
	ret
.L2:
	addi	a0,a1,2
	j	.L1
	.size	doop, .-doop
	.ident	"GCC: (crosstool-NG 1.26.0_rc1) 10.2.0"


In [103]:
%%file /tmp/memset.c
#include <stddef.h>
const int myflags[] = {1 << 0, 1 << 1, 1 << 2};

typedef struct {
    size_t ind;
    int *dst;
    size_t n;
} config;

void mymemset(config *cfg) {
    int color = myflags[cfg->ind];
    int *dst = cfg->dst;
    for(size_t i = 0; i < cfg->n; i++)
        dst[i] = color;
}



Overwriting /tmp/memset.c


In [104]:
! gcc -W -c -S -o /tmp/memset.s -O1   /tmp/memset.c

In [105]:
! /home/philip/Downloads/lowrisc-toolchain-gcc-rv32imcb-x86_64-20250710-1/bin/riscv32-unknown-elf-gcc -W -c -S -o /tmp/memset.s -O1   /tmp/memset.c

In [106]:
! cat /tmp/memset.s

	.file	"memset.c"
	.option nopic
	.attribute arch, "rv32i2p0_m2p0_c2p0"
	.attribute unaligned_access, 0
	.attribute stack_align, 16
	.text
	.align	1
	.globl	mymemset
	.type	mymemset, @function
mymemset:
	lw	a5,0(a0)
	slli	a4,a5,2
	lui	a5,%hi(.LANCHOR0)
	addi	a5,a5,%lo(.LANCHOR0)
	add	a5,a5,a4
	lw	a2,0(a5)
	lw	a5,4(a0)
	lw	a4,8(a0)
	beq	a4,zero,.L1
	li	a4,0
.L3:
	sw	a2,0(a5)
	addi	a4,a4,1
	addi	a5,a5,4
	lw	a3,8(a0)
	bgtu	a3,a4,.L3
.L1:
	ret
	.size	mymemset, .-mymemset
	.globl	myflags
	.section	.rodata
	.align	2
	.set	.LANCHOR0,. + 0
	.type	myflags, @object
	.size	myflags, 12
myflags:
	.word	1
	.word	2
	.word	4
	.ident	"GCC: (crosstool-NG 1.26.0_rc1) 10.2.0"


In [63]:
%%file /tmp/mov42.s
.include "/tmp/knuckle.s"
    .text
    .globl  myfunc
kd_entry myfunc, "(= true true)"
    addi    sp, sp, -4       # make room on the stack
    li      t0, 42           # t0 ← 42
    sw      t0, 0(sp)        # [sp] = 42
kd_exit myfunc_end,  "(= (select ram32 sp) (_ bv42 32))"
    ret

Overwriting /tmp/mov42.s


In [72]:
! python3 -m kdrag.contrib.pcode --langid="RISCV:LE:32:default" --asm /home/philip/Downloads/lowrisc-toolchain-gcc-rv32imcb-x86_64-20250710-1/bin/riscv32-unknown-elf-as /tmp/mov42.s

Processing /tmp/mov42.s with language ID RISCV:LE:32:default using assembler /home/philip/Downloads/lowrisc-toolchain-gcc-rv32imcb-x86_64-20250710-1/bin/riscv32-unknown-elf-as
Constructing Trace Fragments...
Executing 0x400000/4: addi sp,sp,-0x4 at (4194304, 0) PCODE IMARK ram[400000:4]
Executing 0x400000/4: addi sp,sp,-0x4 at (4194304, 1) PCODE unique[580:4] = 0xfffffffc
Executing 0x400000/4: addi sp,sp,-0x4 at (4194304, 2) PCODE sp = sp + unique[580:4]
Executing 0x400004/4: li t0,0x2a at (4194308, 0) PCODE IMARK ram[400004:4]
Executing 0x400004/4: li t0,0x2a at (4194308, 1) PCODE unique[580:4] = 0x2a
Executing 0x400004/4: li t0,0x2a at (4194308, 2) PCODE t0 = unique[580:4]
Executing 0x400008/4: sw t0,0x0(sp) at (4194312, 0) PCODE IMARK ram[400008:4]
Executing 0x400008/4: sw t0,0x0(sp) at (4194312, 1) PCODE unique[600:4] = 0x0
Executing 0x400008/4: sw t0,0x0(sp) at (4194312, 2) PCODE unique[4180:4] = sp + unique[600:4]
Executing 0x400008/4: sw t0,0x0(sp) at (4194312, 3) PCODE *[ram]un

# ghost call
Figuring out how to access members of structs, offsets etc.
Shouldn't change state, can read it. Changed real state is thrown away (stored as var?). Changed ghost state is not.
How to setup frame?
More like a subroutine, since we odn't setup a stack (?).
In ghost code, read from named coroutine params.
Or give fresh
Know the abi somehow?

```
struct biz
get param(&biz) {
    return biz->a->b->c;
}
```

My dishonesty about ram32 ram64 rax bites now?
(RAX reg) (RDI reg)
%rdi == (select32 reg RDI)
(store32 reg RDI 0x42)
`kd_reify "memstate_name"` # freeze current state into ghost.
`kd_assign "memstate" "(store RAX  )"
`kd_ghost_call "res_memstate" "myfun" " "`
`kd_assign  "foo"  "res_memstate"

# read write
I should unpack the state datatype. No reason to bring datatypes into it.
Could use store_view instead of explciit write field?
Jankier

# verilog / sail
Pcode is not that trustworthy. Vet against other semantics.
(And why not just use those semantics instead ?)


https://docs.pydrofoil.org/en/latest/scripting-api.html

rpython
https://mssun.github.io/rpython-by-example/
So rpython itself is a bit like cython? But not exporting python libs. Using rpython on it's own like this is perhaps odd


smtlib interpreter in rpython?
Lean expr interpreter?



In [1]:
! nix build github:rpypkgs/rpypkgs#pydrofoil-riscv

34;1m1/18/19 built, 159 copied (1102.1/1102.7 MiB), 270.7 MiB DL] building pydrofoilm

In [3]:
! nix run github:rpypkgs/rpypkgs#pydrofoil-riscv -- --version

pydrofoil-riscv 0.0.1-alpha0 (Sail model version: 0.5-273-g87f8bb34d6)


In [ ]:
%%file /tmp/hello.py

def entry_point(argv):
    print "Hello, World!"
    return 0

# The target function is the main function of a RPython program. It takes
# command line arguments as inputs and return an entry point function.
def target(*args):
    return entry_point

# python properties


# Inline asm Reg names
Also memory locations?



In [4]:
import z3
z3.parse_smt2_string(r"""
(declare-const %rdi (_ BitVec 64))
(assert (= %rdi (_ bv0 64)))
""")


[%rdi == 0]

In [ ]:
%%file /tmp/hello.c
#include <stddef.h>
__asm__(".include \"knuckle.S\"");
__asm__("kd_prelude \"(declare-const x (_ BitVec 64))\"");
int foo(int x){
    __asm__("kd_entry myentry \"true\"");
    __asm__("kd_assign x %0" : : "r"(x));

    int y = x + 42;
    __asm__("kd_exit myentry \"(assert (= %0 (bvadd x (_ bv42 64)))\"" : : "r"(y));
    return y;
}


Overwriting /tmp/hello.c


In [9]:
! gcc -O1 -S -c /tmp/hello.c -o /tmp/hello.S && cat /tmp/hello.S

	.file	"hello.c"
	.text
	.globl	foo
	.type	foo, @function
foo:
.LFB0:
	.cfi_startproc
	endbr64
#APP
# 3 "/tmp/hello.c" 1
	kd_prelude "(assert (= %edi 42))"
# 0 "" 2
#NO_APP
	ret
	.cfi_endproc
.LFE0:
	.size	foo, .-foo
	.ident	"GCC: (Ubuntu 13.3.0-6ubuntu2~24.04) 13.3.0"
	.section	.note.GNU-stack,"",@progbits
	.section	.note.gnu.property,"a"
	.align 8
	.long	1f - 0f
	.long	4f - 1f
	.long	5
0:
	.string	"GNU"
1:
	.align 8
	.long	0xc0000002
	.long	3f - 2f
2:
	.long	0x3
3:
	.align 8
4:


# CN
https://github.com/rems-project/cn
https://rems-project.github.io/cn-tutorial/getting-started/tutorials/welcome/
https://www.galois.com/articles/escaping-isla-nublar-coming-around-to-llms-for-formal-methods

# cfg reconstruction

Takes VCs

Any unreached specstmt, warn
reachable_from

Join specstmts

Print CFG as ascii?
Output it?



In [ ]:

reachable = set().union(*[vc.trace for vc in vcs])

# Hyperproperties / Relational

Synchro points?

entry exit labels pairs.
kd_sync

kd_rel entry exit property
kd_rel entry_exit property

kd_channels
monotonic channels?

Finely interleaved product programs

A syncrhonzination / scheudling program?
ram_2

Multiram executor? Channels

Same program twice is easy to synchrnoize.
High low variables

havoc statement
kd_havoc_ghost ghost_var
kd_havoc_reg reg
kd_havoc_mem memexpr  
interrupt havocs touched stuff every cycle. So we do want to enable havor

https://docs.certora.com/en/latest/docs/cvl/statements.html

kd_join
collect up all trace fragements with common start and ends.
Maybe I should anyhow?

Useful for the using assembly as spec idea.

trace fragment bundle. Yikes

thread_id as a way of knowing which program you're in?





# Unicorn
Running stuff in pcode2c was clutch.
What about just a nice running environemnt that ingests asmspec.
Unicorn, gdb, qemu. Something.





chat
missing vex optimziationd via z3

p4-=

Concerns - user to 


# New Sleigh
https://github.com/nneonneo/ghidra-wasm-plugin/tree/master

https://spinsel.dev/2020/06/17/ghidra-brainfuck-processor-1.html

3 roads to go down: https://github.com/rbran/sleigh-rs/tree/main try to reuse this sleigh parser of nebulous quality, try to hack onto ghidra's sleigh parser https://github.com/NationalSecurityAgency/ghidra/blob/master/Ghidra/Features/Decompiler/src/decompile/cpp/slghparse.y , or try to rebuild the grammar in a custom parser (edited) 
Ok actually a fourth is to consume the compiled .sla files. I don't even really know how their format works
Why? Well, because using ghidra off the shelf only lets you turn concrete bits into pcode (edited) 
But the Sleigh format is kind of more like the analog of sail. You could use it to compile or assemble forward (which ghidra does use it to generate assemblers but I haven't seen that rewrapped to be usable anywhere except in the gui)
https://github.com/NationalSecurityAgency/ghidra/blob/master/Ghidra/Framework/SoftwareModeling/src/main/java/ghidra/app/plugin/assembler/sleigh/SleighAssemblerBuilder.java


https://github.com/CTSRD-CHERI/ghidra cheri modelled in ghidra?
https://github.com/Cherified/cheriot-abstract-spec  https://github.com/Cherified/cheriot-program-logic
https://github.com/CTSRD-CHERI/ghidra/blob/morello/main/Ghidra/Processors/AARCH64/data/languages/AARCH64_MorelloInstructions.sinc

Hmm generate sleigh from yosys akin to how C backend works maybe?


In [4]:
! mkdir /tmp/Hack || mkdir /tmp/Hack/data

mkdir: cannot create directory ‘/tmp/Hack’: File exists
mkdir: cannot create directory ‘/tmp/Hack/data’: File exists


In [51]:
%%file /tmp/Hack/Module.manifest
<module>
  <name>Hack</name>
  <description>nand2tetris Hack CPU</description>
  <version>1.0</version>
</module>


Overwriting /tmp/Hack/Module.manifest


In [52]:
%%file /tmp/Hack/data/hack.pspec
<?xml version="1.0" encoding="UTF-8"?>
<processor_spec>
  <programcounter register="PC"/>
  <default_symbols>
    <function name="entry" entry="true" address="0x0000"/>
  </default_symbols>
</processor_spec>

Writing /tmp/Hack/data/hack.pspec


In [53]:
%%file /tmp/Hack/data/hack.ldefs
<?xml version="1.0" encoding="UTF-8"?>
<language_definitions>
  <language processor="Hack" endian="big" size="16" variant="default"
            version="1" slafile="hack.sla" processorspec="hack.pspec"
            description="Hack (nand2tetris) 16-bit"/>
  <compiler name="default" spec="hack.cspec" id="default"/>
</language_definitions>

Writing /tmp/Hack/data/hack.ldefs


In [ ]:
%%file /tmp/Hack/data/hack.slaspec
define endian=big;
define alignment=2;

define space ram     type=ram_space   size=2  default;
define space register type=register_space size=1;

define register offset=0x00 size=2  [PC D A];
define token instr(16)
  b15 = (15,15)             # 0 = A-instr, 1 = C-instr
  cxx = (13,14)             # the conventional "11" in Hack C-instr
  a   = (12,12)
  c1  = (11,11)
  c2  = (10,10)
  c3  = (9,9)
  c4  = (8,8)
  c5  = (7,7)
  c6  = (6,6)
  d1  = (5,5)               # A dest
  d2  = (4,4)               # D dest
  d3  = (3,3)               # M dest
  j1  = (2,2)
  j2  = (1,1)
  j3  = (0,0)
  aimm = (0,14)             # 15-bit literal for A-instruction
;
#OP1: "#"aimm    is b15=0; aimm			{ tmp:2 = aimm; export tmp; }
:a_inst aimm is b15=0 & aimm {
  A = aimm;          # zero-extend 15-bit literal
  PC = PC + 2;
}

Overwriting /tmp/Hack/data/hack.slaspec


In [95]:
! /home/philip/Downloads/ghidra_11.4.2_PUBLIC_20250826/ghidra_11.4.2_PUBLIC/support/sleigh /tmp/Hack/data/hack.slaspec

openjdk version "21.0.8" 2025-07-15
OpenJDK Runtime Environment (build 21.0.8+9-Ubuntu-0ubuntu124.04.1)
OpenJDK 64-Bit Server VM (build 21.0.8+9-Ubuntu-0ubuntu124.04.1, mixed mode)
INFO  Using log config file: jar:file:/home/philip/Downloads/ghidra_11.4.2_PUBLIC_20250826/ghidra_11.4.2_PUBLIC/Ghidra/Framework/Generic/lib/Generic.jar!/generic.log4j.xml (LoggingInitialization)  
INFO  Using log file: /home/philip/.config/ghidra/ghidra_11.4.2_PUBLIC/application.log (LoggingInitialization)  


In [108]:
import kdrag.contrib.pcode as pcode
import pypcode
lang = pypcode.Arch("Hack", "/tmp/Hack/data/hack.ldefs").languages[0]
ctx = pypcode.Context(lang)
tx = ctx.disassemble(bytes.fromhex("1f"))
print(tx.instructions[0].mnem)
print(pcode.pretty_insn(tx.instructions[0]))
tx = ctx.translate(bytes.fromhex("1f"))
print(tx.ops)
pcode.pretty_op(tx.ops[1])
tx.ops[1].inputs[0].size

a_inst
0x0/2: a_inst 0x1f00
[IMARK ram[0:2], A = 0x1f00, PC = PC + 0x2]


2

# sail
Give sail another shake. Comparing and constrasting to sleigh is kind of interesting.



# vec
https://en.algorithmica.org/hpc/simd/

In [13]:
%%file /tmp/sum.c
//#pragma GCC target("avx2")
const int n = 1e5;
int a[n], s = 0;

int main() {
    for (int t = 0; t < 100000; t++)
        for (int i = 0; i < n; i++)
            s += a[i];

    return 0;
}

Overwriting /tmp/sum.c


In [15]:
! g++ -O3 /tmp/sum.c -o /tmp/sum && time /tmp/sum


real	0m0.526s
user	0m0.522s
sys	0m0.001s


In [14]:
! g++ -march=native -O3 /tmp/sum.c -o /tmp/sum && time /tmp/sum


real	0m0.137s
user	0m0.136s
sys	0m0.000s


In [ ]:
! python3 -m pypcode  

usage: pypcode [-h] [-l] [-o OFFSET] [-s LENGTH] [-i MAX_INSTRUCTIONS] [-b]
               langid binary [base]
pypcode: error: the following arguments are required: langid, binary


In [16]:
! g++ -O3 /tmp/sum.c -o /tmp/sum.S -S -masm=intel && head -40 /tmp/sum.S

	.file	"sum.c"
	.intel_syntax noprefix
	.text
	.section	.text.startup,"ax",@progbits
	.p2align 4
	.globl	main
	.type	main, @function
main:
.LFB0:
	.cfi_startproc
	endbr64
	lea	rdx, a[rip]
	movd	xmm0, DWORD PTR s[rip]
	lea	rcx, 400000[rdx]
.L2:
	movdqa	xmm1, XMMWORD PTR [rdx]
	mov	eax, 100000
	.p2align 4,,10
	.p2align 3
.L3:
	paddd	xmm0, xmm1
	paddd	xmm0, xmm1
	sub	eax, 2
	jne	.L3
	add	rdx, 16
	cmp	rcx, rdx
	jne	.L2
	movdqa	xmm1, xmm0
	xor	eax, eax
	psrldq	xmm1, 8
	paddd	xmm0, xmm1
	movdqa	xmm1, xmm0
	psrldq	xmm1, 4
	paddd	xmm0, xmm1
	movd	DWORD PTR s[rip], xmm0
	ret
	.cfi_endproc
.LFE0:
	.size	main, .-main
	.globl	s


In [18]:
! g++ -march=native -O3 /tmp/sum.c -o /tmp/sum.S -S -masm=intel && head -60 /tmp/sum.S

	.file	"sum.c"
	.intel_syntax noprefix
	.text
	.section	.text.startup,"ax",@progbits
	.p2align 4
	.globl	main
	.type	main, @function
main:
.LFB0:
	.cfi_startproc
	endbr64
	lea	rdx, a[rip]
	vmovd	xmm0, DWORD PTR s[rip]
	lea	rcx, 400000[rdx]
.L2:
	vmovdqa32	zmm1, ZMMWORD PTR [rdx]
	mov	eax, 100000
	.p2align 4
	.p2align 3
.L3:
	vpaddd	zmm0, zmm0, zmm1
	sub	eax, 2
	vpaddd	zmm0, zmm0, zmm1
	jne	.L3
	add	rdx, 64
	cmp	rcx, rdx
	jne	.L2
	vmovdqa	ymm1, ymm0
	vextracti32x8	ymm0, zmm0, 0x1
	xor	eax, eax
	vpaddd	ymm1, ymm1, ymm0
	vmovdqa	xmm0, xmm1
	vextracti128	xmm1, ymm1, 0x1
	vpaddd	xmm0, xmm0, xmm1
	vpsrldq	xmm1, xmm0, 8
	vpaddd	xmm0, xmm0, xmm1
	vpsrldq	xmm1, xmm0, 4
	vpaddd	xmm0, xmm0, xmm1
	vmovd	DWORD PTR s[rip], xmm0
	vzeroupper
	ret
	.cfi_endproc
.LFE0:
	.size	main, .-main
	.globl	s
	.bss
	.align 4
	.type	s, @object
	.size	s, 4
s:
	.zero	4
	.globl	a
	.align 64
	.type	a, @object
	.size	a, 400000
a:
	.zero	400000
	.ident	"GCC: (Ubuntu 13.3.0-6ubuntu2~24.04) 13.3.0"
	.section	.note.GNU-stac

In [21]:
%%file /tmp/support.cpp
#include <iostream>
using namespace std;

int main() {
    cout << __builtin_cpu_supports("sse") << endl;
    cout << __builtin_cpu_supports("sse2") << endl;
    cout << __builtin_cpu_supports("avx") << endl;
    cout << __builtin_cpu_supports("avx2") << endl;
    cout << __builtin_cpu_supports("avx512f") << endl;

    return 0;
}

Writing /tmp/support.cpp


In [23]:
! g++ -O3 /tmp/support.cpp -o /tmp/support && /tmp/support

8
16
512
1024
32768


In [25]:
! cat /proc/cpuinfo

processor	: 0
vendor_id	: AuthenticAMD
cpu family	: 25
model		: 116
model name	: AMD Ryzen 7 PRO 7840U w/ Radeon 780M Graphics
stepping	: 1
microcode	: 0xa704107
cpu MHz		: 4707.835
cache size	: 1024 KB
physical id	: 0
siblings	: 16
core id		: 0
cpu cores	: 8
apicid		: 0
initial apicid	: 0
fpu		: yes
fpu_exception	: yes
cpuid level	: 16
wp		: yes
flags		: fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 ht syscall nx mmxext fxsr_opt pdpe1gb rdtscp lm constant_tsc rep_good amd_lbr_v2 nopl xtopology nonstop_tsc cpuid extd_apicid aperfmperf rapl pni pclmulqdq monitor ssse3 fma cx16 sse4_1 sse4_2 x2apic movbe popcnt aes xsave avx f16c rdrand lahf_lm cmp_legacy svm extapic cr8_legacy abm sse4a misalignsse 3dnowprefetch osvw ibs skinit wdt tce topoext perfctr_core perfctr_nb bpext perfctr_llc mwaitx cpb cat_l3 cdp_l3 hw_pstate ssbd mba perfmon_v2 ibrs ibpb stibp ibrs_enhanced vmmcall fsgsbase bmi1 avx2 smep bmi2 erms invpcid cqm rdt_a avx512f 